In [ ]:
import os.path
import subprocess
from io import StringIO

import pandas as pd

from kcmc_instance import KCMC_Instance

from tqdm.notebook import tqdm
    

list_serialized_instances = pd.read_csv('/data/instances.csv', header=None)[0].tolist()

SER_INSTANCE = list_serialized_instances[0]
obj_instance = KCMC_Instance(SER_INSTANCE)

K = 3
M = 2

W1 = 0.1
W2 = 0.45
W3 = 0.45

TIMEOUT = 120
VERBOSITY = 250
POPULATION = 10
SELECTION = 4
MUTATION = 0.05

obj_instance

In [ ]:
result_optimizer = subprocess.run(list(map(str, [
    "timeout", TIMEOUT,
    "/builds/optimizer_gupta_exact", VERBOSITY,
    POPULATION, SELECTION, MUTATION,
    K, M, W1, W2, W3, SER_INSTANCE
])), capture_output=True)
    
stdout = result_optimizer.stdout.decode()
stderr = result_optimizer.stderr.decode()

assert stderr.strip().lower().endswith('exiting gracefully...')
stderr

In [ ]:
df = pd.read_csv(StringIO(stdout), sep='\t', dtype=str)

col_int = ['GEN_IT', 'TIMESTAMP_MS', 'SIZE']
df.loc[:, col_int] = df[col_int].apply(lambda col: col.astype(int))
df.loc[:, 'FITNESS'] = df['FITNESS'].astype(float)

df['GEN_IT'].max(), df['GEN_IT'].max()/TIMEOUT

In [ ]:
import plotly.graph_objects as go

idf = df#.iloc[:-1]

fig = go.Figure()

fig.add_trace(go.Scatter(x=idf['TIMESTAMP_MS'], y=idf['FITNESS']*10,
                         mode='lines', name='FITNESS x10'))
fig.add_trace(go.Scatter(x=idf['TIMESTAMP_MS'], y=idf['SIZE'],
                         mode='lines', name='SOLUTION SIZE'))

In [ ]:
df.sort_values('FITNESS', ascending=False).head(10)

In [ ]:
df.sort_values('SIZE').head(10)

In [ ]:
obj_instance.plot(
    labels=True, minimal=True,
    installation={
        f'i{i}': '1'
        for i, v in enumerate(df.loc[62, 'CHROMOSSOME'])
        if v == '1'
    })